# Featurizer
이 노트북은 `TigerGraph`에 저장된 그래프의 피처 엔지니어링 작업에 `pyTigerGraph`를 사용하는 방법을 보여줍니다.



**NOTE**: 현재 ML Workbench에서 제공하는 모든 피처을 사용하려면 데이터베이스를 한 번만 활성화(activate)해야 합니다. ML Workbench on Cloud를 사용하는 경우 activator가 포함되며 아래 셀을 실행(먼저 주석 해제)하여 활성화할 수 있습니다. 다른 버전의 Workbench의 경우 https://act.tigergraphlabs.com 에서 activater를 다운로드할 수 있습니다. 자세한 지침은 해당 웹사이트에도 포함되어 있습니다.

In [ ]:
# 아래 주석을 제거하고 필요한 정보를 입력하십시오. 자세한 지침은 https://act.tigergraphlabs.com을 참조하십시오.
# !mlwb activate [데이터베이스 주소] -u [사용자 이름] -p [비밀번호] -s [비밀]

## 데이터베이스 연결 

이 `TigerGraphConnection` 클래스는 `TigerGraph` 데이터베이스에 대한 연결을 나타냅니다. 내부 적으로는 데이터베이스와 통신하는 데 필요한 정보가 저장됩니다. 꽤 많은 데이터베이스 작업을 수행할 수 있습니다. 자세한 내용은 해당 [문서](https://docs.tigergraph.com/pytigergraph/current/intro/) 를 참조하십시오.

**Note**: 2022년 7월 5일 이후에 생성된 TG 클라우드 DB는 사용자 이름/비밀번호 대신 Secret이 필요합니다. 그렇지 않으면 비워 둘 수 있습니다.


In [ ]:
from pyTigerGraph import TigerGraphConnection

In [ ]:
conn = TigerGraphConnection(
    host="http://127.0.0.1", # 데이터베이스 서버의 주소로 변경
    graphname="Cora",
    username="tigergraph",
    password="tigergraph",
    gsqlSecret="" # 2022년 7월 5일 이후에 생성된 TG 클라우드 DB에는 user/pass 대신 secret이 필요합니다.  
)

<span style="color:red">아래 셀의 주석을 제거하고 토큰 인증이 활성화된 경우 토큰을 가져오고 설정하기 위해 실행합니다.</span>. 
* 이것은 tgcloud의 모든 데이터베이스에 필요합니다.
* `<secret>`은 사용자 암호입니다. 자세한 내용은 https://docs.tigergraph.com/tigergraph-server/current/user-access/managing-credentials#_secrets 를 참조하세요.
* secret을 모르는 경우  `secret=conn.createSecret()`를 사용하여 secret을 만들 수 있습니다.

In [ ]:
#conn.getToken(<secret>)

In [ ]:
# 그래프 스키마 및 기타 정보.
print(conn.gsql("ls"))

In [ ]:
# 모든 정점 유형에 대한 정점 수
conn.getVertexCount('*')

In [ ]:
# 특정 유형의 정점 수
conn.getVertexCount("Paper")

In [ ]:
# 모든 유형의 가장자리 수
conn.getEdgeCount()

In [ ]:
# 특정 유형의 모서리 수
conn.getEdgeCount("Cite")

## 피처 엔지니어링 
ML Workbench에는 피처 엔지니어링 작업을 수행하기 위한 몇 가지 그래프 알고리즘이 포함되어 있습니다. 주요 피처은 다음과 같습니다:

1. `listAlgorithm()`: 알고리즘 클래스(예: 중심성:Centrality)를 입력으로 받으면 지정된 범주에 대해 사용 가능한 알고리즘을 인쇄합니다. 그렇지 않으면 사용 가능한 모든 알고리즘을 인쇄합니다. 
2. `installAlgorithm()`: 알고리즘의 이름을 입력으로 가져와 알고리즘이 아직 설치되지 않은 경우 설치합니다.
3. `runAlgorithmm()`: 알고리즘 이름, 스키마 유형(예: 정점/간선, 기본적으로 정점), 속성 이름(결과를 데이터베이스에 속성으로 저장해야 하는 경우) 및 스키마 유형 이름 목록(속성을 저장해야 하는 정점/간선 목록, 기본적으로 모든 정점/간선에 대한 것)을 가져옵니다.

In [ ]:
f = conn.gds.featurizer()

In [ ]:
f.listAlgorithms()

### GDS 라이브러리 그래프 알고리즘의 실행 예(Examples of running graph algorithms from GDS library)
다음에서는 알고리즘의 각 클래스에 대한 한 가지 예가 제공됩니다. 일부 알고리즘은 정점/간선 별로 피처을 생성합니다. 다른 알고리즘은 그래프에 대한 숫자 또는 통계를 계산합니다. 예를 들어, 공통 이웃(common neighbor) 알고리즘은 두 정점 사이의 공통 이웃 수를 계산합니다.

#### 피처로 Pagerank 가져오기(Get Pagerank as a feature)
pagerank는 GDS 라이브러리의 tg_pagerank https://github.com/tigergraph/gsql-graph-algorithms/blob/master/algorithms/Centrality/pagerank/global/unweighted/tg_pagerank.gsql 라는 중심성(centrality) 알고리즘 클래스에서 사용할 수 있습니다.

In [ ]:
f.installAlgorithm("tg_pagerank")

In [ ]:
params = {'v_type': 'Paper', 'e_type': 'Cite', 'max_change': 0.001, 'max_iter': 25, 'damping': 0.85,
          'top_k': 10, 'print_accum': True, 'result_attr': '', 'file_path': '', 'display_edges': False}

f.runAlgorithm(
    'tg_pagerank', 
    params=params, 
    feat_name="pagerank",
    global_schema=False, # 속성을 추가하기 위해 전역 스키마 변경이 필요한 경우 True로 설정합니다.
    timeout=2147480, 
    sizeLimit=2000000
)


#### 최대 독립 셋(Maximal Independent Set) 실행
최대 독립 셋 알고리즘은 GDS 라이브러리의 tg_maximal_indep_set https://github.com/tigergraph/gsql-graph-algorithms/blob/master/algorithms/Classification/maximal_independent_set/deterministic/tg_maximal_indep_set.gsql 이라는 분류(classification) 알고리즘 클라스에서 사용할 수 있습니다.

In [ ]:
f.installAlgorithm("tg_maximal_indep_set")

In [ ]:
params = {'v_type': 'Paper', 'e_type': 'Cite',
          'max_iter': 100, 'print_accum': False, 'file_path': ''}

f.runAlgorithm('tg_maximal_indep_set', params=params)


#### Louvain을 피처으로 가져오기(Get Louvain as a feature)
Louvain 알고리즘은 GDS 라이브러리의 tg_louvain https://github.com/tigergraph/gsql-graph-algorithms/blob/master/algorithms/Community/louvain/tg_louvain.gsql 이라는 탐지(detection) 알고리즘 클라스에서 사용할 수 있습니다.

In [ ]:
f.installAlgorithm(query_name='tg_louvain')

In [ ]:
params = {'v_type': 'Paper', 'e_type': ['Cite', 'reverse_Cite'], 'wt_attr': "",
          'max_iter': 10, 'result_attr': "cid", 'file_path': "", 'print_info': True}

f.runAlgorithm(
    'tg_louvain', 
    params, 
    feat_name="cid",
    global_schema=False # If a global schema change is needed to add an attribute, set it to True.
)


#### 피처으로 fastRP 얻기(Get fastRP as a feature)
fastRP 알고리즘은 GDS 라이브러리의 g_fastRP https://github.com/tigergraph/gsql-graph-algorithms/blob/master/algorithms/GraphML/Embeddings/FastRP/tg_fastRP.gsql 이라는 임베딩(embedding) 알고리즘 클라스에서 사용할 수 있습니다.

In [ ]:
f.installAlgorithm(
    "tg_fastRP", 
    global_change = False # If a global schema change is needed to add an attribute, set it to True.
)

In [ ]:
params = {'v_type': 'Paper', 'e_type': ['Cite', 'reverse_Cite'], 'weights': '1,1,2', 'beta': -0.85, 'k': 3, 'reduced_dim': 128,
          'sampling_constant': 1, 'random_seed': 42, 'print_accum': False, 'result_attr': "", 'file_path': ""}
f.runAlgorithm(
    'tg_fastRP', 
    params, 
    feat_name="fastrp_embedding"
)


#### 단일 소스 노드에서 너비 우선 검색 알고리즘 실행(Run Breadth-First Search Algorithm from a single source node)
너비 우선 검색(Breadth-First Search) 알고리즘은 GDS 라이브러리의 tg_bfs https://github.com/tigergraph/gsql-graph-algorithms/blob/master/algorithms/Path/bfs/tg_bfs.gsql 이라는 경로(Path) 알고리즘 클라스에서 사용할 수 있습니다.

In [ ]:
f.installAlgorithm(query_name='tg_bfs')

In [ ]:
params = {'v_type': 'Paper', 'e_type': ['Cite', 'reverse_Cite'], 'max_hops': 10, "v_start": ("2180", "Paper"),
          'print_accum': False, 'result_attr': "", 'file_path': "", 'display_edges': False}

f.runAlgorithm('tg_bfs', params, feat_name="bfs")


#### 두 정점 사이의 공통 이웃 수를 계산(Calculates the number of common neighbors between two vertices)
공통 이웃(common neighbors) 알고리즘은 GDS 라이브러리의 tg_common_neighbors https://github.com/tigergraph/gsql-graph-algorithms/blob/master/algorithms/Topological%20Link%20Prediction/common_neighbors/tg_common_neighbors.gsql 이라는 토폴로지 링크 예측(Topological Link Prediction) 알고리즘 클라스에서 사용할 수 있습니다.

In [ ]:
f.installAlgorithm(query_name='tg_common_neighbors')  

In [ ]:
params = {"a": ("2180", "Paper"), "b": ("431", "Paper"),
          "e_type": "Cite", "print_res": True}

f.runAlgorithm('tg_common_neighbors', params)


### 사용자 정의 쿼리(User Defined Query) 사용

In [ ]:
user_defined_query1 = '''CREATE QUERY user_defined_query1() FOR GRAPH Cora { 
  PRINT "user_defined_query works!"; 
}'''

print(user_defined_query1)

In [ ]:
outFileName="./user_defined_query1.gsql"
outFile=open(outFileName, "w")
outFile.write(user_defined_query1)
outFile.close()

In [ ]:
f.installAlgorithm(query_name="user_defined_query1", query_path="./user_defined_query1.gsql" )

In [ ]:
f.runAlgorithm(query_name="user_defined_query1", custom_query=True)